# Carregando os dados

In [21]:
import pandas as pd

In [22]:
%%time
transactions_2009_2010 = pd.read_excel('../online_retail_II.xlsx',sheet_name="Year 2009-2010", engine='openpyxl')
transactions_2010_2011= pd.read_excel('../online_retail_II.xlsx',sheet_name="Year 2010-2011", engine='openpyxl')

CPU times: total: 1min 37s
Wall time: 1min 38s


In [23]:
# Concatenar os DataFrames (empilhar as linhas)
transactions = pd.concat([transactions_2009_2010, transactions_2010_2011], ignore_index=True)

# Exibir as primeiras linhas do DataFrame combinado
print(transactions.shape)

(1067371, 8)


In [24]:
transactions.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


# Limpeza de Dados

In [25]:
# Verificando valores ausentes
transactions.isna().sum()

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64

# Análise de Dados Básicos

In [42]:
# Remove as transações canceladas
transactions = transactions[~transactions['Invoice'].astype(str).str.contains('C', na=False)]

# Agrupoar por produto e somar as quantidades vendidas
top_products = transactions.groupby('Description')['Quantity'].sum().reset_index()

# Ordenar do mais vendido para o menos vendido
top_products = top_products.sort_values(by='Quantity', ascending=False)

# Mostra os 10 produtos mais vendidos
print(top_products.head(10))

                             Description  Quantity
4389  WHITE HANGING HEART T-LIGHT HOLDER     58793
4495   WORLD WAR 2 GLIDERS ASSTD DESIGNS     55202
719                  BROCADE RING PURSE      48384
2738    PACK OF 72 RETRO SPOT CAKE CASES     46755
279        ASSORTED COLOUR BIRD ORNAMENT     45356
147          60 TEATIME FAIRY CAKE CASES     36502
2736  PACK OF 60 PINK PAISLEY CAKE CASES     31940
2154             JUMBO BAG RED RETROSPOT     30807
3973                SMALL POPCORN HOLDER     29824
4056      STRAWBERRY CERAMIC TRINKET BOX     27128


In [18]:
transactions_grouped.shape

(53628, 2)

In [19]:
transactions_grouped.head()

,0,1
0,489434,0 85048 1 79323P 2 79323W 3 2204...
1,489435,8 22350 9 22349 10 22195 11 2235...
2,489436,12 48173C 13 21755 14 21754 15 ...
3,489437,31 22143 32 22145 33 22130 34 ...
4,489438,54 21329 55 21252 56 21100 57 ...
